<h1>🦜LangChain</h1>
<h2>Summarize a YouTube Video + Count token Usage</h2><img src="yt.png" />
<style>
h1 {text-align: center;}
h2 {text-align: center;}
div {text-align: center;}
</style>

### Objectives : 
    summarize a video - LangChain YouTube Loader + OpenAI
    count token usage - LangChain
    translate the summary to German - Helsinki-NLP/opus-mt-en-de

In [42]:
url = 'https://python.langchain.com/docs/modules/model_io/models/llms/how_to/token_usage_tracking'

In [33]:
#!pip install langchain
#!pip install sacremoses
from tqdm import tqdm

In [43]:
import asyncio # for callback only
from langchain.callbacks import get_openai_callback
from langchain.document_loaders import YoutubeLoader
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [45]:
loader = YoutubeLoader(video_id="Onzd5QxKaGQ")
result = loader.load()

<class 'list'>


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(result)

In [35]:
texts

[Document(page_content="gitco News special coverage of Bitcoin 2023 is brought to you by coin payments crypto payments made easy I'm Michelle McCrory we are at Bitcoin 2023 the largest Bitcoin conference in the world and we are kicking off our coverage with arguably a legend in the Bitcoin space Michael Saylor executive chairman and founder of microstrategy a prolific investor in Bitcoin best-selling author and one of the biggest proponents of Bitcoin Michael so good to have you with us thank you for having me well Michael I actually had the pleasure of first meeting you at Bitcoin 2021 we had an interview prior to that and then I had the pleasure of chatting to you face to face in Bitcoin 2021 and you said then that Bitcoin is going to be the emerging store of value Asset in the 21st century the dominant emerging store value Asset in the 21st century in terms of how Bitcoin is being deployed as a store of value Asset are you where you thought we would be two years on yeah actually um 

In [22]:
llm=OpenAI(temperature =0,openai_api_key=OPENAI_API_KEY,)

# ££

In [46]:
with get_openai_callback() as cb:
    chain =load_summarize_chain(llm, chain_type="stuff", verbose=True)
    tx = chain.run(texts[:2])
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(tx)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"gitco News special coverage of Bitcoin 2023 is brought to you by coin payments crypto payments made easy I'm Michelle McCrory we are at Bitcoin 2023 the largest Bitcoin conference in the world and we are kicking off our coverage with arguably a legend in the Bitcoin space Michael Saylor executive chairman and founder of microstrategy a prolific investor in Bitcoin best-selling author and one of the biggest proponents of Bitcoin Michael so good to have you with us thank you for having me well Michael I actually had the pleasure of first meeting you at Bitcoin 2021 we had an interview prior to that and then I had the pleasure of chatting to you face to face in Bitcoin 2021 and you said then that Bitcoin is going to be the emerging store of value Asset in the 21st century the dominant emerging store value Asset in the 21st century in terms 

In [13]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# input_text = f"translate {tx} to German"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# # Adjust the parameters for generating longer translations
# outputs = model.generate(
#     input_ids,
#     max_length=300,  # Increase the maximum length of the generated translation
#     num_beams=10,   # Increase the number of beams for diverse translations
#     early_stopping=True  # Stop generation when all beams finished
# )

# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# german_translation = decoded_output.split(".")[0].strip()
# print(german_translation)

Michael Saylor, Vorstandsvorsitzender und Gründer von Microstrategy, ist ein prominentes Persönlichkeit in der Bitcoin-Sektor


In [36]:
from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")

input_text = f"translate {tx} to de"

input_ids = tokenizer.encode(input_text, return_tensors="pt")
translated_ids = model.generate(input_ids, max_length=1024, num_return_sequences=1, early_stopping=True)

german_translation = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
# Split at the first occurrence of punctuation followed by a space to get the first complete sentence
german_sentence = german_translation.split(" .", 1)[0] + " ."
print(german_sentence)



Übersetzen Michael Saylor, Executive Chairman und Gründer von Microstrategy, ist eine prominente Figur im Bitcoin Raum. Er glaubt, dass Bitcoin ist die dominierende aufstrebenden Speicher von Wertvermögen im 21. Jahrhundert, übertrifft alle anderen wichtigen Asset-Klassen seit August 2020. Microstrategy hat in 140.000 Bitcoin als Reserve-Asset investiert, und während 24 börsennotierten Unternehmen folgen, Saylor glaubt, die Buchhaltung von Bitcoin ist eine Herausforderung für mehr Unternehmen, es zu übernehmen. zu de .
